In [2]:
import torch
from torch import nn
import numpy as np
import os
import time
import codecs
import re
import json
import pandas as pd
import glob

In [4]:
#get filenames
rootdir = "MyFitbitData5/Mad/Physical Activity"
regex = re.compile('heart_rate-2021')
filenames = os.listdir(rootdir)
heartRates = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            heartRates = np.append(heartRates, file)


#read files for hr data
inputs = np.empty((0,3))
dateTime = []
minInputs = np.empty((0,3))
for x in range(np.size(heartRates)):
    filename = 'MyFitbitData5/Mad/Physical Activity/' + heartRates[x]
    data = []
    
    with open(filename) as json_file:
        data = json.load(json_file)
        inputs = np.empty((len(data), 3))
        for y in range(len(data)):
            temp = data[y]["dateTime"][9:] 
            inputs[y][0] = int(temp[:2])*3600  + int(temp[3:5])*60 + int(temp[6:9])
            inputs[y][1] = int(data[y]["value"]["bpm"])
            inputs[y][2] = x+1
            if inputs[y][2] == 0:
                print("Hello")
        minInputs = np.vstack((minInputs, inputs))


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))

for x in range(len(minInputs) - 1):
    if x == 82529:
        print(minInputs[x])
        print(minInputs[x+1])
    if minInputs[x][0] <= minInputs [x+1][0]:
        minInputs[x][0] = minInputs[x+1][0] - minInputs[x][0]
    else:
        minInputs[x][0] = 86400 - minInputs[x][0] + minInputs[x+1][0]
        
        
        
    '''if minInputs[x][0] > 10:
        tot = int(minInputs[x][0]/10)
        print(tot)
        print(minInputs[x])
        
        minInputs[x][0] = minInputs[x][0]%10
        
        for y in range(tot):    
            intervals = minInputs[0:x+y, 0:1]
            currentInterval = np.sum(intervals)           
            minInputs = np.insert(minInputs, [x+1],[currentInterval, (minInputs[x][1]+minInputs[x+1][1])/2,int((currentInterval+((tot-y)*10))/86400)], axis=0)'''
        
print(minInputs)

minInputs = np.delete(minInputs, len(minInputs)-1, 0)
train_data_normalized2 = scaler.fit_transform(np.transpose(minInputs)[1] .reshape(-1,1))
scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized1 = scaler.fit_transform(np.transpose(minInputs)[0] .reshape(-1,1))

scaler = MinMaxScaler(feature_range=(-1,1))

train_data_normalized3 = scaler.fit_transform(np.transpose(minInputs)[2] .reshape(-1,1))

train_data_normalized = np.hstack((train_data_normalized1, train_data_normalized2))

train_data_normalized = np.hstack((train_data_normalized, train_data_normalized3))
train_data_normalized =torch.FloatTensor(train_data_normalized)
print(numOfInputs)
inputs = train_data_normalized.view(-1, 303, train_data_normalized.shape[1])
print(inputs)
numOfInputs = np.size(np.transpose(minInputs)[0])


[1.8332e+04 4.0000e+01 8.0000e+00]
[1.8342e+04 4.1000e+01 8.0000e+00]
[[1.5000e+01 8.2000e+01 1.0000e+00]
 [1.0000e+01 8.1000e+01 1.0000e+00]
 [5.0000e+00 7.4000e+01 1.0000e+00]
 ...
 [5.0000e+00 5.8000e+01 8.8000e+01]
 [5.0000e+00 5.7000e+01 8.8000e+01]
 [8.2795e+04 5.8000e+01 8.8000e+01]]


NameError: name 'numOfInputs' is not defined

In [ ]:
rootdir = "MyFitbitData5/Mad/Nutrition"

regex = re.compile('food_logs-0')
filenames = os.listdir(rootdir)
foodNames = []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            foodNames = np.append(foodNames, file)

#127
for x in range(np.size(foodNames)):
    filename = 'MyFitbitData5/Mad/Nutrition/' + foodNames[x]
    data = []
    print(filename)
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        
        for x in range(np.size(roughdata)):
            
            cal = roughdata[x]["loggedFood"]["calories"]
            del roughdata[x]["logId"]
            del roughdata[x]["loggedFood"]
            del roughdata[x]["favorite"]
            if "nutritionalValues" in roughdata[x].keys():                
                del roughdata[x]["nutritionalValues"]

                
            roughdata[x]["calories"] = cal
        data = np.append(data, roughdata)
calories = data[0]["calories"]
totCalories = np.empty((0,1))
dates = 
for x in range(len(data)-1):
    if data[x+1]["logDate"] == data[x]["logDate"]:
        calories += data[x+1]["calories"]
        if x == len(data)-2:
            totCalories = np.vstack((totCalories, calories))
    else:
        totCalories = np.vstack((totCalories, calories))
        calories = data[x+1]["calories"]
        
    
totCalories[10][0] += 1050

print(totCalories)

In [ ]:
rootdir = "MyFitbitData5/Mad/personal & Account"
regex = re.compile('weight-2021-03-05')
filenames = os.listdir(rootdir)
weights= []
for root, dirs, files in os.walk(rootdir):
    for file in files:
        if regex.match(file):
            weights = np.append(weights, file)

print(weights)            
for x in range(np.size(weights)):
    filename = 'MyFitbitData5/Mad/personal & Account/' + weights[x]
    data = []
    weight = []
    fat = []
    with open(filename) as json_file:
        roughdata = json.load(json_file)
        for x in range(np.size(roughdata)):
            del roughdata[x]["logId"]
            del roughdata[x]["time"]
            del roughdata[x]["source"]
            del roughdata[x]["bmi"]
            weight = np.append(weight, roughdata[x]["weight"])
            fat = np.append(fat, roughdata[x]["fat"])

        data = np.append(data, roughdata)

weightfat = data

fat = np.insert(fat, 4, [])

def caloriesBurned(weight, fat):
    fatMass = []
    leanMass = []
    for x in range(len(weight)):
        fatMass = np.append(fatMass, weight[x]*fat[x]/100)
        leanMass = np.append(leanMass, weight[x] - fatMass[x])
    calories = [] 
    print(fatMass)

    for x in range(len(weight) - 1):   
        calories = np.append(calories, -(fatMass[x]-fatMass[x+1])*442.39)
        if leanMass[x] >= leanMass[x+1]:
            calories[x] -= (leanMass[x] - leanMass[x+1])*70
        else:
            calories[x] -= (leanMass[x]-leanMass[x+1])*265
    return calories
        
    
calories = caloriesBurned(weight, fat)

for x in range(len(calories)):
    calories[x] = totCalories[x] - calories[x]

#calories = np.append(calories, [2000,2000, 2000])
print(calories)


In [ ]:
y_actual = []

for x in range(numOfInputs):
    interval = minInputs[x][0]
    day = minInputs[x][2]
   
    intervalCalorie = calories[int(day-1)]/86400 * interval        
    y_actual = np.append(y_actual, intervalCalorie)


scaler = MinMaxScaler(feature_range=(-1,1))

normalized_y = torch.FloatTensor(scaler.fit_transform(y_actual .reshape(-1,1)))
outputs = normalized_y.view(-1, 303, normalized_y.shape[1])
print(normalized_y)
print(outputs)



In [ ]:




class RNN(nn.Module):
    def __init__(self, input_size = 3, hidden_size = 5, output_size = 1):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, 1)
        self.linear = nn.Linear(hidden_size, output_size)
        
        self.hidden_cell = (torch.zeros(1,1,self.hidden_size),torch.zeros(1,1,self.hidden_size))
        
        
    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq),1,-1), self.hidden_cell)
        predictions = self.linear(lstm_out)
        return predictions
    
n_hidden = 5
lstm = RNN()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.0001)
print(lstm)

In [1]:
epochs = 100
print(normalized_y[0:3, 0:3])
print(lstm(inputs[0]))
print(np.sum(scaler.inverse_transform(outputs[0])))

for i in range(epochs):
    for x in range(len(inputs)-1):    
        optimizer.zero_grad()
        lstm.hidden_cell = (torch.zeros(1,1, lstm.hidden_size), torch.zeros(1,1,lstm.hidden_size))
        y_pred = lstm(inputs[x])
        single_loss = loss_function(y_pred.float(), outputs[x].float())
        single_loss.backward()
        optimizer.step()
        
    
    
    
    print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')
            

NameError: name 'normalized_y' is not defined

In [57]:
y_pred = lstm(inputs[len(inputs)-1])
single_loss = loss_function(y_pred.float(), outputs[len(inputs) -1].float())
print(single_loss)
yo = y_pred.detach().numpy()

woo =scaler.inverse_transform(np.transpose(np.transpose(yo)[0]))
print(np.sum(woo))
#print(woo)
print(np.sum(scaler.inverse_transform(outputs[len(inputs) -1])))

tensor(1.4312e-06, grad_fn=<MseLossBackward>)
33.841354
53.471649915315595


C:\Users\User\anaconda3\lib\site-packages\torch\nn\modules\loss.py:431: UserWarning: Using a target size (torch.Size([303, 1])) that is different to the input size (torch.Size([303, 1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
